In [ ]:
import numpy as np
import cupy as cp
import sys
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import h5py
sys.path.insert(0, '..')
from utils import *

In [ ]:
path = f'/data/vnikitin/ESRF/ID16A/20240924_rec_ca/data/'
with  h5py.File(f'{path}/data_atomium.h5','r') as fid:
    data = fid[f'/exchange/data'][:].astype('float32')
    ref0 = fid[f'/exchange/data_white0'][:].astype('float32')
    ref1 = fid[f'/exchange/data_white1'][:].astype('float32')
    dark = fid[f'/exchange/data_dark'][:].astype('float32')
    theta = fid[f'/exchange/theta'][:].astype('float32')
    
    shifts = fid[f'/exchange/shifts'][:].astype('float32')
    attrs = fid[f'/exchange/attrs'][:].astype('float32')
    
    z1 = fid['/exchange/z1'][0]
    detector_pixelsize = fid['/exchange/detector_pixelsize'][0]
    focusdetectordistance = fid['/exchange/focusdetectordistance'][0]
    energy = fid['/exchange/energy'][0]


In [ ]:
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
z2 = focusToDetectorDistance-z1
distances = (z1*z2)/focusToDetectorDistance
magnification = focusToDetectorDistance/z1
voxelsize = np.abs(detector_pixelsize/magnification)  # object voxel size


# Pre-processing

In [ ]:
import cupyx.scipy.ndimage as ndimage
def remove_outliers(data, dezinger, dezinger_threshold):    
    res = data.copy()
    w = [dezinger,dezinger]
    for k in range(data.shape[0]):
        data0 = cp.array(data[k])
        fdata = ndimage.median_filter(data0, w)
        print(np.sum(np.abs(data0-fdata)>fdata*dezinger_threshold))
        res[k] = np.where(np.abs(data0-fdata)>fdata*dezinger_threshold, fdata, data0).get()
    return res

dark = np.mean(dark,axis=0)
data -= dark
ref0-=dark
ref1-=dark

data[data<0]=0
ref0[ref0<0] = 0
ref1[ref1<0] = 0
data[:,1320//3:1320//3+25//3,890//3:890//3+25//3] = data[:,1280//3:1280//3+25//3,890//3:890//3+25//3]
ref0[:,1320//3:1320//3+25//3,890//3:890//3+25//3] = ref0[:,1280//3:1280//3+25//3,890//3:890//3+25//3]
ref1[:,1320//3:1320//3+25//3,890//3:890//3+25//3] = ref1[:,1280//3:1280//3+25//3,890//3:890//3+25//3]

data = remove_outliers(data, 3, 0.8)    
ref0 = remove_outliers(ref0, 3, 0.8)    
ref1 = remove_outliers(ref1, 3, 0.8)    

mshow(data[0],mshow)

In [ ]:
# adjust amps

In [ ]:
ref0=np.mean(ref0,axis=0)
cnorm = np.mean(ref0)
data=data/cnorm/(attrs[:,1]/attrs[0,1])[:,np.newaxis,np.newaxis]
ref0/=cnorm
ref1/=cnorm

In [ ]:
path_out = f'/data/vnikitin/ESRF/ID16A/20240924_rec_ca/data/'
with  h5py.File(f'{path_out}/data_atomium.h5','a') as fid:
    try:
        del fid[f'/exchange/pdata']
        del fid[f'/exchange/pref0']
        del fid[f'/exchange/pref1']
    except:
        pass
    fid.create_dataset(f'/exchange/pdata',data=data)     
    fid.create_dataset(f'/exchange/pref0',data=ref0)     
    fid.create_dataset(f'/exchange/pref1',data=ref1)     